In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('full_data_shuffle_24_04_2997.csv')
data.columns = ['sample_num', str(0), str(1), str(2), str(3), str(4), str(5),str(6),str(7),str(8),str(9),str(10),str(11),str(12),str(13), 'gesture']

#data = data.drop(['Unnamed: 0'], axis = 1)
d_t = pd.DataFrame(StandardScaler().fit_transform(data.drop(['sample_num', 'gesture'], axis = 1)))
d_t['gesture'] = data['gesture']
d_t['sample_num'] = data['sample_num']
data = d_t
#data = data.sample(frac=1).reset_index(drop=True)
data.shape
len(data['sample_num'].unique())

############################################################

import numpy as np
import random
np_lst = []
sample_nums_rand = data['sample_num'].unique()
random.shuffle(sample_nums_rand)
for n in sample_nums_rand:
    sample = data[data['sample_num'] == n].drop(['sample_num', 'gesture'], axis = 1)
    np_lst.append(sample.values)
    
data_reshaped = np.array(np_lst).reshape(( 2997,1,14, 100))
np_lst

#############################################################

from keras.utils import to_categorical
y = data[['gesture', 'sample_num']]
l = []
for i in sample_nums_rand:
    s_y = y[y['sample_num']==i]
    if(s_y['gesture'].iloc[0] == 0):
        l.append(0)
    else:
        l.append(1)
#print(data['gesture'])      
y = to_categorical(np.array(l).reshape((len(l), )))
y

#################################################################

X =  data_reshaped


Using TensorFlow backend.


In [2]:
import numpy as np
from sklearn.model_selection import KFold

def get_cv_score(est, checkpointer, X, y, n_epoch = 30, val_split = 0.1):
    kf = KFold(n_splits=10)
    acc_row = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #checkpointer = ModelCheckpoint(filepath='./checkpoit.h5', verbose=1, save_best_only=True)
        est.fit(X_train, y_train, batch_size = 64, epochs = n_epoch, 
                                    verbose = 2,validation_split = val_split,
                                    callbacks=[checkpointer])
        est.load_weights('./checkpoit.h5')
        loss, acc = est.evaluate(X_test,  y_test, verbose=2)
        acc_row.append(acc)
    acc_row.append(np.mean(acc_row))
    return  acc_row

In [3]:
from EEGModels import EEGNet_old
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import metrics
model  = EEGNet_old(nb_classes=2, Chans = 14, Samples = 100, regRate = 0.0001,
           dropoutRate = 0.25, kernels = [(2, 32), (8, 4)], strides = (2, 4))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['categorical_accuracy'])
checkpointer = ModelCheckpoint(filepath='./checkpoit.h5', verbose=1, save_best_only=True)
acc_row = get_cv_score(model, checkpointer, X, y, n_epoch = 500, val_split = 0.1)
table = []
table.append(acc_row)
res = pd.DataFrame(table)
res

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 2427 samples, validate on 270 samples
Epoch 1/500

Epoch 00001: val_loss improved from inf to 0.76325, saving model to ./checkpoit.h5
2427/2427 - 4s - loss: 0.8314 - categorical_accuracy: 0.4990 - val_loss: 0.7632 - val_categorical_accuracy: 0.5111
Epoch 2/500

Epoch 00002: val_loss improved from 0.76325 to 0.76128, saving model to ./checkpoit.h5
2427/2427 - 0s - loss: 0.7801 - categorical_accuracy: 0.5056 - val_loss: 0.7613 - val_categorical_accuracy: 0.5185
Epoch 3/500

Epoch 00003: val_loss improved from 0.76128 to 0.75896, saving model to ./checkpoit.h5
2427/2427 - 0s - loss: 0.7680 - categorical_accuracy: 0.5014 - val_loss: 0.7590 - val_categorical_accuracy: 0.4963
Epoch 4/500

Epoch 00004: val_loss improved from 0.75896 to 0.75607, saving model to ./checkpoit.h5
2427/2427 - 0s - loss: 0.7703 - categorical_accuracy: 0.4969 - val_loss: 0.7561 - val_categorical_accuracy: 0.4815
Epoch 5/500

Epo

Epoch 42/500

Epoch 00042: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6540 - categorical_accuracy: 0.6761 - val_loss: 0.7596 - val_categorical_accuracy: 0.4889
Epoch 43/500

Epoch 00043: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6575 - categorical_accuracy: 0.6737 - val_loss: 0.7533 - val_categorical_accuracy: 0.5000
Epoch 44/500

Epoch 00044: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6558 - categorical_accuracy: 0.6613 - val_loss: 0.7371 - val_categorical_accuracy: 0.5111
Epoch 45/500

Epoch 00045: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6570 - categorical_accuracy: 0.6753 - val_loss: 0.7450 - val_categorical_accuracy: 0.5074
Epoch 46/500

Epoch 00046: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6562 - categorical_accuracy: 0.6737 - val_loss: 0.7671 - val_categorical_accuracy: 0.5074
Epoch 47/500

Epoch 00047: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6523 - cat

Epoch 88/500

Epoch 00088: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6037 - categorical_accuracy: 0.7285 - val_loss: 0.8347 - val_categorical_accuracy: 0.5519
Epoch 89/500

Epoch 00089: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6075 - categorical_accuracy: 0.7355 - val_loss: 0.8156 - val_categorical_accuracy: 0.5296
Epoch 90/500

Epoch 00090: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6286 - categorical_accuracy: 0.7252 - val_loss: 0.8146 - val_categorical_accuracy: 0.5481
Epoch 91/500

Epoch 00091: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5982 - categorical_accuracy: 0.7351 - val_loss: 0.8201 - val_categorical_accuracy: 0.5185
Epoch 92/500

Epoch 00092: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5881 - categorical_accuracy: 0.7412 - val_loss: 0.8610 - val_categorical_accuracy: 0.5407
Epoch 93/500

Epoch 00093: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.6036 - cat

Epoch 134/500

Epoch 00134: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5314 - categorical_accuracy: 0.7878 - val_loss: 0.9082 - val_categorical_accuracy: 0.5667
Epoch 135/500

Epoch 00135: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5634 - categorical_accuracy: 0.7787 - val_loss: 0.8868 - val_categorical_accuracy: 0.5444
Epoch 136/500

Epoch 00136: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5596 - categorical_accuracy: 0.7849 - val_loss: 0.9597 - val_categorical_accuracy: 0.5333
Epoch 137/500

Epoch 00137: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5650 - categorical_accuracy: 0.7651 - val_loss: 0.9141 - val_categorical_accuracy: 0.5519
Epoch 138/500

Epoch 00138: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5412 - categorical_accuracy: 0.7895 - val_loss: 0.9491 - val_categorical_accuracy: 0.5593
Epoch 139/500

Epoch 00139: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5475

Epoch 180/500

Epoch 00180: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.4833 - categorical_accuracy: 0.8245 - val_loss: 1.0389 - val_categorical_accuracy: 0.5519
Epoch 181/500

Epoch 00181: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.4918 - categorical_accuracy: 0.8257 - val_loss: 1.0315 - val_categorical_accuracy: 0.5519
Epoch 182/500

Epoch 00182: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5106 - categorical_accuracy: 0.8142 - val_loss: 1.0007 - val_categorical_accuracy: 0.5407
Epoch 183/500

Epoch 00183: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.4703 - categorical_accuracy: 0.8414 - val_loss: 1.0638 - val_categorical_accuracy: 0.5333
Epoch 184/500

Epoch 00184: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.4810 - categorical_accuracy: 0.8414 - val_loss: 1.0746 - val_categorical_accuracy: 0.5370
Epoch 185/500

Epoch 00185: val_loss did not improve from 0.73094
2427/2427 - 0s - loss: 0.5787

...

Epoch 419/500

Epoch 00419: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3551 - categorical_accuracy: 0.9164 - val_loss: 1.4337 - val_categorical_accuracy: 0.4704
Epoch 420/500

Epoch 00420: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3576 - categorical_accuracy: 0.9262 - val_loss: 1.4791 - val_categorical_accuracy: 0.4815
Epoch 421/500

Epoch 00421: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3451 - categorical_accuracy: 0.9267 - val_loss: 1.4869 - val_categorical_accuracy: 0.4667
Epoch 422/500

Epoch 00422: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3560 - categorical_accuracy: 0.9213 - val_loss: 1.5030 - val_categorical_accuracy: 0.4667
Epoch 423/500

Epoch 00423: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3598 - categorical_accuracy: 0.9135 - val_loss: 1.5291 - val_categorical_accuracy: 0.4741
Epoch 424/500

Epoch 00424: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3653

Epoch 465/500

Epoch 00465: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3556 - categorical_accuracy: 0.9250 - val_loss: 1.5949 - val_categorical_accuracy: 0.4926
Epoch 466/500

Epoch 00466: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.4048 - categorical_accuracy: 0.9052 - val_loss: 1.5982 - val_categorical_accuracy: 0.4963
Epoch 467/500

Epoch 00467: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.4702 - categorical_accuracy: 0.8735 - val_loss: 1.4753 - val_categorical_accuracy: 0.4741
Epoch 468/500

Epoch 00468: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3963 - categorical_accuracy: 0.9061 - val_loss: 1.4777 - val_categorical_accuracy: 0.4889
Epoch 469/500

Epoch 00469: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.4416 - categorical_accuracy: 0.8879 - val_loss: 1.3665 - val_categorical_accuracy: 0.4926
Epoch 470/500

Epoch 00470: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.3630

Epoch 10/500

Epoch 00010: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6626 - categorical_accuracy: 0.6580 - val_loss: 0.7473 - val_categorical_accuracy: 0.5074
Epoch 11/500

Epoch 00011: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6662 - categorical_accuracy: 0.6576 - val_loss: 0.7502 - val_categorical_accuracy: 0.5148
Epoch 12/500

Epoch 00012: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6651 - categorical_accuracy: 0.6535 - val_loss: 0.7387 - val_categorical_accuracy: 0.5037
Epoch 13/500

Epoch 00013: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6584 - categorical_accuracy: 0.6646 - val_loss: 0.7441 - val_categorical_accuracy: 0.4889
Epoch 14/500

Epoch 00014: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6617 - categorical_accuracy: 0.6658 - val_loss: 0.7470 - val_categorical_accuracy: 0.5037
Epoch 15/500

Epoch 00015: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6557 - cat

Epoch 56/500

Epoch 00056: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6237 - categorical_accuracy: 0.7070 - val_loss: 0.7928 - val_categorical_accuracy: 0.5481
Epoch 57/500

Epoch 00057: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6309 - categorical_accuracy: 0.7000 - val_loss: 0.8092 - val_categorical_accuracy: 0.4815
Epoch 58/500

Epoch 00058: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6076 - categorical_accuracy: 0.7219 - val_loss: 0.7809 - val_categorical_accuracy: 0.5519
Epoch 59/500

Epoch 00059: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.5960 - categorical_accuracy: 0.7318 - val_loss: 0.8041 - val_categorical_accuracy: 0.5519
Epoch 60/500

Epoch 00060: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6110 - categorical_accuracy: 0.7268 - val_loss: 0.8185 - val_categorical_accuracy: 0.5370
Epoch 61/500

Epoch 00061: val_loss did not improve from 0.72876
2427/2427 - 0s - loss: 0.6322 - cat

KeyboardInterrupt: 

In [4]:
from EEGModels import EEGNet
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import metrics
model  = EEGNet(2, Chans = 14, Samples = 100, F1 = 62, D = 6, F2 = 90, dropoutRate = 0.5, kernLength = 200, norm_rate = 0.4, dropoutType = 'SpatialDropout2D', regularizator = 0.0001)
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['categorical_accuracy'])
checkpointer = ModelCheckpoint(filepath='./checkpoit.h5', verbose=1, save_best_only=True)
acc_row = get_cv_score(model, checkpointer, X, y, n_epoch = 500, val_split = 0.1)
table = []
table.append(acc_row)
res = pd.DataFrame(table)
res

ValueError: Negative dimension size caused by subtracting 14 from 1 for 'depthwise_conv2d/depthwise' (op: 'DepthwiseConv2dNative') with input shapes: [?,1,14,62], [14,1,62,6].

In [ ]:
from EEGModels import   ShallowConvNet
from EEGModels import   DeepConvNet
from EEGModels import EEGNet
from EEGModels import EEGNet_SSVEP

model  = EEGNet_SSVEP(2, Chans = 14, Samples = 100,
       dropoutRate = 0.5)
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['categorical_accuracy'])
checkpointer = ModelCheckpoint(filepath='./checkpoit.h5', verbose=1, save_best_only=True)
acc_row = get_cv_score(model, checkpointer, X, y, n_epoch = 50, val_split = 0.1)
table = []
table.append(acc_row)
res = pd.DataFrame(table)
res